# Cleaning data with PySpark - Datacamp 
### Notes by César Muro Cabral

* Data Cleaning: Prepare raw data for use in data processin pipelines.  
* Possible tasks in data cleaning: Reformatting or replacing text, performing calculations, removing garbage or incomple data  
* Problems with typical data systems: Performance, organizing data flow
* Advantages of spark: scalable, powerful framework for data handling


**Spark Schemas**  
Spark provides a built-in ability to validate datasets with schemas
* Define the format of a DataFrame
* May contain different types of data
* Can filter garbage during import
* Improves read performance  

Example of import schema:    
import pyspark.sql.types    
peopleSchema=Structype([ StructField('name',StringType(),True), StructField('age',IntegerType(),True), StructField('city',StringType(),True)])        

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.3.2-bin-hadoop2'

Defining a schema

Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

 *   Name
 *   Age
 *   City

The Name and City columns are StringType() and the Age column is an IntegerType()

In [3]:
# Import the pyspark.sql.types library
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city',StringType(),False)
])

**Immutability and lazy processing**  
Python variables:
* Mutable
* Flexibility
* Potential for issues with concurrency
* Likely adds complexity    
  
Inmutable variables (as spark dataframes) are: 
* Spark takes advantage of data immutability to efficiently share / create new data representations throughout the cluster.
* A component of functional programming
* Defined once
* Unable to be directly modified
* Recreated if reassigned
* Able to be shared efficiently   
Spark is efficent because of something called lazy processing:  
* Very little actually happens until an action is performed
* We only update the instructions (aka, transformations) for what we wanted Spark to do. 


Using lazy processing

Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to Spark not performing any transformations until an action is requested.

For this exercise, we'll be defining a Data Frame (aa_dfw_df) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparent.

In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [6]:
import pyspark.sql
from pyspark.sql.functions import lower
# Load the CSV file
path_file='cleaning_data_with_pyspark_datasets/AA_DFW_2017_Departures_Short.csv'
aa_dfw_df = spark.read.format('csv').options(Header=True).load(path_file)

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
|       01/01/2017|         0060|                          149|    mia|
|       01/01/2017|         0071|                          203|    lax|
|       01/01/2017|         0074|                           76|    mem|
|       01/01/2017|         0081|                          123|    den|
|       01/01/2017|         0089|                          161|    slc|
|       01/01/2017|         0096|                           84| 

**Understanding Parquet**  
Difficulties with CSV files:  
* No defined schema
* Nested data requieres special handling
* Encoding format limited
* Slow to parse. If no schema is defined, all data must be read before a schema can be inferred
* Files cannot be filtered (no "predicate pushdown")  
* Any intermediate use requiere redefining schema  
  
The parquet format:  
* A columnar data format developed for use in any Hadoop based system.  
* The parquet format is structured with data accessible in chunks, allowing efficien read/write operations without processing the entire file 
* Supports predicate pushdown functionality, providing performance improvement
* Automatically include schema information and handle data encoding
* Interact with parquet files is very straightforward: 
df=spark.read.format('parquet').load('filename.parque')    
df=spark.read.parque('filename.parquet')   
* Writing parquet files
df.write.format('parquet').save('filename.parquet')  
  
Parquet and SQL:  
* Parquet as backing stores for SparkSql operations:  
flight_df=spark.read.parquet('flights.parquet')    
flights_df.createOrReplaceTempView('flights')    



Saving a DataFrame in Parquet format

When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The Parquet format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets. 

In [7]:
df1=spark.read.csv('cleaning_data_with_pyspark_datasets/AA_DFW_2016_Departures_Short.csv',header=True,inferSchema=True)
df2=spark.read.csv('cleaning_data_with_pyspark_datasets/AA_DFW_2017_Departures_Short.csv',header=True,inferSchema=True)
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

df1 Count: 140604
df2 Count: 139358
279962


SQL and Parquet

Parquet files are perfect as a backing data store for SQL queries in Spark. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

In [14]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')
flights_df=flights_df.withColumnRenamed('Actual elapsed time (Minutes)','flight_duration')

# Register the temp table
flights_df.createOrReplaceTempView('flights')
# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 151


## Manipulating DataFrames in the real world
Common DataFrame transformations:  
* Filter/Where:  
voter_df.filter(voter_df.date>'1/1/2019')    
voter_df.filter(voter_df.name.like('M%'))  
* select():  
voter_df.select(voter_df.name)  
* withColumn()  
voter_df.withColumn('year',voter_df.date.year)  
* drop() method to drop a column:  
voter_df.drop('unused_column')
  
Filterin data:  
* remove nulls, remove odd entries, split data from combined sources
* negate with ∼  
voter_df.where(∼ voter_df._c1.isNull())  
  
Column string transformations: 
* Contained in pyspark.sql.functions  
import pyspark.sql.functions as F  
voter_df.withColumn('upper',F.upper('name'))  
* Can create intermediary columns  
voter_df.withColumn('splits',F.split('name',' '))  
* Can cast to other types  
voter_df.withColumn('year',voter_df['_c4'].cast(IntegerType()))  
  
ArrayType() column functions to interact with ArrayType() 
.size(<column>) - returns lenght of arrayType()   
.getItem(<Index>) - used to retrieve a specific item at index of list column    

Filtering column content with Python

You've looked at using various operations on DataFrame columns - now you can modify a real dataset. The DataFrame voter_df contains information regarding the voters on the Dallas City Council from the past few years. This truncated DataFrame contains the date of the vote being cast and the name and position of the voter. Your manager has asked you to clean this data so it can later be integrated into some desired reports. The primary task is to remove any null entries or odd characters and return a specific set of voters where you can validate their information.

This is often one of the first steps in data cleaning - removing anything that is obviously outside the format. For this dataset, make sure to look at the original data and see what looks out of place for the VOTER_NAME column.

In [15]:
import pyspark.sql.functions as F 
voter_df=spark.read.csv('cleaning_data_with_pyspark_datasets/DallasCouncilVoters.csv',header=True,inferSchema=True)
# Show the distinct VOTER_NAME entries
voter_df.select(voter_df['VOTER_NAME']).distinct().show(40, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

# Show the distinct VOTER_NAME entries again
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Modifying DataFrame columns

Previously, you filtered out any rows that didn't conform to something generally resembling a name. Now based on your earlier work, your manager has asked you to create two new columns - first_name and last_name. She asks you to split the VOTER_NAME column into words on any space character. You'll treat the last word as the last_name, and all other words as the first_name. You'll be using some new functions in this exercise including .split(), .size(), and .getItem(). The .getItem(index) takes an integer value to return the appropriately numbered item in the column. The functions .split() and .size() are in the pyspark.sql.functions library.

Please note that these operations are always somewhat specific to the use case. Having your data conform to a format often matters more than the specific details of the format. Rarely is a data cleaning task meant just for one person - matching a defined format allows for easier sharing of the data later (ie, Paul doesn't need to worry about names - Mary already cleaned the dataset).

In [17]:
import pyspark.sql.functions as F 
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show()

C:\spark-3.3.2-bin-hadoop2\python\pyspark\sql\column.py:419: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


+----------+-------------+-------------------+----------+---------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|
+----------+-------------+-------------------+----------+---------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|   Arnold|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|   Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|  McGough|
|02/08/2017|Councilmember|       Lee Kleinman|       Lee| Kleinman|
|02/08/2017|Councilmember|      Sandy Greyson|     Sandy|  Greyson|
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|  

**Conditional DataFrame column operations**    
Often you'll want to conditionally change some aspect of the contents. Spark provides some built in conditional clauses which act similar to an if / then / else statement in a traditional programming environment. Conditional clauses:  
* Inline version of if/then/else  
* .when(\<if condition\>,\<then x\>)    
* .otherwise() is like else   
E.g.  
df.select(df.Name,df.Age,F.when(df.Age >=18,"Adult"))  
df.select(df.Name, df.Age,.when(df.Age>=18,"Adult"),.when(df.Age<18,"Minor"))  

In [23]:
# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE=='Councilmember', F.rand()))

# Show some of the DataFrame rows, noting whether the when clause worked
voter_df.show()

+----------+-------------+-------------------+----------+---------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|         random_val|
+----------+-------------+-------------------+----------+---------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates| 0.9041701597594819|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston| 0.3838436737981362|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|               null|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano| 0.6400750494567863|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas|   0.08297711851833|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|   Arnold| 0.9950740741701446|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|   Griggs| 0.9987681684165531|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|  McGough| 0.8068266660595841|
|02/08/2017|Councilmember|      

When / Otherwise

This requirement is similar to the last, but now you want to add multiple values based on the voter's position. Modify your voter_df DataFrame to add a random number to any voting member that is defined as a Councilmember. Use 2 for the Mayor and 0 for anything other position.

The voter_df Data Frame is defined and available to you. The pyspark.sql.functions library is available as F. You can use F.rand() to generate the random value.

In [24]:
# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand())
                               .when(voter_df.TITLE=='Mayor', 2).otherwise(0))
                               

# Show some of the DataFrame rows
voter_df.show()

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val==0).show()

+----------+-------------+-------------------+----------+---------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|first_name|last_name|         random_val|
+----------+-------------+-------------------+----------+---------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|    Gates| 0.7211796904493689|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| Kingston| 0.9183424191141313|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael| Rawlings|                2.0|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|  Medrano| 0.9216375914846742|
|02/08/2017|Councilmember|       Casey Thomas|     Casey|   Thomas| 0.7926435834353253|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|   Arnold| 0.7895781689958441|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|   Griggs| 0.9560541178448072|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|  McGough| 0.6042029353120666|
|02/08/2017|Councilmember|      

**User defined functions or UDF**  
It is a python method that the user writes to perform a specific bit of logic.  
* Wrapped via the pyspark.sql.functions.udf  
* Stored as variable  
def reverseString(mystr):  
   return mystr[::-1]  
Wrap the function and store as a variable 
   
udfReverseString=udf(reverseString,StringType())  
  
Use with Spark:  
user_df=user_df.withColumn('ReverseName',idfReverseString(user_df.Name))    

In [25]:
import pyspark.sql.functions as F 
voter_df=spark.read.csv('cleaning_data_with_pyspark_datasets/DallasCouncilVoters.csv',header=True,inferSchema=True)
# Show the distinct VOTER_NAME entries
voter_df.select(voter_df['VOTER_NAME']).distinct().show(40, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

import pyspark.sql.functions as F 
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))


def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names[:-1])

# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

# Show the DataFrame
voter_df.show()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

C:\spark-3.3.2-bin-hadoop2\python\pyspark\sql\column.py:419: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


+----------+-------------+-------------------+--------------------+----------+---------+---------------------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|first_and_middle_name|
+----------+-------------+-------------------+--------------------+----------+---------+---------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|          Jennifer S.|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|            Philip T.|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|           Michael S.|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|                 Adam|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Thomas|                Casey|
|02/08/2017|Councilmember|Carolyn King Arnold|[Carolyn, King, A...|   Carolyn|   Arnold|         Carolyn King|
|

**Partitioning and lazy processing**  
* DataFrames are broken up into partitions, that can be automatically defined, enlarged, schrunk. 
* Partition size can vary
* Each partition is handled independently
* In Spark, any transformation operation is lazy; its more like a recipe than a command. .withColumn(), .select()  
* Nothing is actually done until an action is performed. .count(), .write()  
* Transformations can be re-ordered for best performance  
  
Relational databases tend to have a field used to identify the row. These IDs are typically an integer that increases in value, is sequential, and most importantly unique.  
* Not very in parallel. In a distributed platform such as Spark, it creates some undue bottlenecks
* Spark has a built-in function designed to provided an integer ID that increases in value and is unique-
* pyspark.sql.functions.monotonically_increasing_id()
* Integer (64-bit), increases in value, unique
* Not necessarily sequential (gaps exist)  
* Completely parallel



Adding an ID Field

When working with data, you sometimes only want to access certain fields and perform various operations. In this case, find all the unique voter names from the DataFrame and add a unique ID number. Remember that Spark IDs are assigned based on the DataFrame partition - as such the ID values may be much greater than the actual number of rows in the DataFrame.

With Spark's lazy processing, the IDs are not actually generated until an action is performed and can be somewhat random depending on the size of the dataset.

In [36]:
import pyspark.sql.functions as F 
voter_df=spark.read.csv('cleaning_data_with_pyspark_datasets/DallasCouncilVoters.csv',header=True,inferSchema=True)
# Show the distinct VOTER_NAME entries
voter_df.select(voter_df['VOTER_NAME']).distinct().show(40, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

# Select all the unique council voters
voter_df = df.select(df["VOTER_NAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Improving Performance  
Caching in spark:  
* Caching in Spark refers to storing the results of a DataFrame in memory or on disk of the processing nodes in a cluster  
* Improves speed on later transformations / actions
* Reduces resource usage  
  
Disadvantages of caching:  
* Very large data sets may not fit in memory
* Local disk based caching may not be a performance improvement
* Cached objects may not be avalaible  
  
When developing Spark tasks:  
* Cache only if you need it 
* Try caching DataFrames at various point and determine if your performance improves  
* Cache in memory and fast SSD / NVMe storage
* Cache to slow local disk if needed
* Use intermediate files!
*  Stop caching when finish  
  
Call .cache() on the DataFrame after action  
Check .is_cached to determine cache status  
Call .unpersist() when finished the DataFrame  


Caching a DataFrame

You've been assigned a task that requires running several analysis operations on a DataFrame. You've learned that caching can improve performance when reusing DataFrames and would like to implement it.

You'll be working with a new dataset consisting of airline departure information. It may have repetitive data and will need to be de-duplicated.

In [30]:
import time
start_time = time.time()

departures_df = spark.read.parquet('AA_DFW_ALL.parquet')
# Add caching to the unique rows in departures_df
departures_df = departures_df.distinct().cache()

# Count the unique rows in departures_df, noting how long the operation takes
print("Counting %d rows took %f seconds" % (departures_df.count(), time.time() - start_time))

# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (departures_df.count(), time.time() - start_time))

Counting 279962 rows took 3.504681 seconds
Counting 279962 rows again took 1.118129 seconds


Removing a DataFrame from cache

You've finished the analysis tasks with the departures_df DataFrame, but have some other processing to do. You'd like to remove the DataFrame from the cache to prevent any excess memory usage on your cluster.

In [31]:
# Determine if departures_df is in the cache
print("Is departures_df cached?: %s" % departures_df.is_cached)
print("Removing departures_df from cache")

# Remove departures_df from the cache
departures_df.unpersist()

# Check the cache status again
print("Is departures_df cached?: %s" % departures_df.is_cached)

Is departures_df cached?: True
Removing departures_df from cache
Is departures_df cached?: False


**Improve import performance**  
Spark clusters are made of two types of processes:  
* Driver process
* Work processes  
  
When importing data to Spark DataFrame it is important to understand how the cluster implements the job. Important parameters:  
* Number of objects (Files, Network location, etc)  
   - More objects better than larger ones
   - Can import vial wildcar
   airport_df=spark.read.csv('airports-*.txt.gz')  
* General size of objects
  - Spark performs better if objects are of similar size

* Well defined schema will drastically improve import performance
   - Avoids reading the data multiple times
   - Provides validation on import  

* How to split objects  
   - Use OS utilities / scripts (split,cut,awk)  
      split -l 10000 -d largefile chunk-
   - Use custom scripts
   - Write out to Parquet     

Converting to a larger number of files with approximately equal quantity of rows lets Spark decide how best to read the data.

File import performance

You've been given a large set of data to import into a Spark DataFrame. You'd like to test the difference in import speed by splitting up the file.

You have two types of files available: departures_full.txt.gz and departures_xxx.txt.gz where xxx is 000 - 013. The same number of rows is split between each file.

In [ ]:
# Import the full and split files into DataFrames
full_df = spark.read.csv('departures_full.txt.gz')
split_df = spark.read.csv('departures_0*.txt.gz')

# Print the count and run time for each DataFrame
start_time_a = time.time()
print("Total rows in full DataFrame:\t%d" % full_df.count())
print("Time to run: %f" % (time.time() - start_time_a))

start_time_b = time.time()
print("Total rows in split DataFrame:\t%d" % split_df.count())
print("Time to run: %f" % (time.time() - start_time_b))

**Cluster configurations**  
* Spark contains many configuration settings
* These can be modified to match needs
* Reading configuration settings: spark.conf.get(\<configuration name\>)
* Writing configuration settings: spark.conf.set(\<configuration name\>)  
  
Spark deployment options:  
  - Single node 
  - Standalone
  - Managed: YARN, Mesos, Kubernetes  

Driver:  
* Task assignment
* Result consolidation
* Shared data access
Tips: 
* Driver nodel should have double memory of the worker
* Fast local storage helpful  
  
Worker: 
* Runs actual tasks
* ideally has all code, data, and resources for a given task
Recommendations:
* More worker nodes is often better than larger workers
* Test to find the balance
* Fast local storage extremely useful

Reading Spark configurations

You've recently configured a cluster via a cloud provider. Your only access is via the command shell or your python code. You'd like to verify some Spark settings to validate the configuration of the cluster.

In [32]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')

# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: pyspark-shell
Driver TCP port: 52729
Number of partitions: 200


Writing Spark configurations

Now that you've reviewed some of the Spark configurations on your cluster, you want to modify some of the settings to tune Spark to your needs. You'll import some data to review that your changes have affected the cluster.

The spark configuration is initially set to the default value of 200 partitions.

In [33]:
departures_df = spark.read.parquet('AA_DFW_ALL.parquet')
# Store the number of partitions in variable
before = departures_df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
departures_df = spark.read.csv('AA_DFW_ALL.parquet').distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % departures_df.rdd.getNumPartitions())

Partition count before change: 4
Partition count after change: 1


**Performance improvements**  
The .explain() method on a dataframe.  
Spark distributes data amongst the various nodes in the cluster. Aside effect of this is what is known as shuffling. It refers to moving data around to various workers to complete a task.  
* Hides complexity from the user
* Can be slow to complete
* Lowers overall throughput
* Is often neccesary component, but try to minimize  
  
How to limit shuffling?  
* Limit use of .repartition(num_partitions). Use .coalesce(num_partitions) instead
* Use care when calling .join(), use .broadcast()  
  
Broadcasting in Spark is a method to provide a copy of an object to each worker.
* It prevents undue / ecess communication between nodes
* Can dastrically speed up .join() operations
Use the .broadcast(DataFrame) method  
from pyspark.sql.functions import broadcast  
combine_df=df_1.join(broadcas(df_2))  


Normal joins

You've been given two DataFrames to combine into a single useful DataFrame. Your first task is to combine the DataFrames normally and view the execution plan.

The DataFrames flights_df and airports_df are available to you.

In [ ]:
# Don't execute, the data frame airports is not provided
# Join the flights_df and aiports_df DataFrames
normal_df = flights_df.join(airports_df, \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan
normal_df.explain()

Using broadcasting on Spark joins

Remember that table joins in Spark are split between the cluster workers. If the data is not local, various shuffle operations are required and can have a negative impact on performance. Instead, we're going to use Spark's broadcast operations to give each node a copy of the specified data.

A couple tips:

    Broadcast the smaller DataFrame. The larger the DataFrame, the more time required to transfer to the worker nodes.
    On small DataFrames, it may be better skip broadcasting and let Spark figure out any optimization on its own.
    If you look at the query execution plan, a broadcastHashJoin indicates you've successfully configured broadcasting

In [ ]:
# Don't execute, the airports data frame is not provided
# Import the broadcast method from pyspark.sql.functions
from pyspark.sql.functions import broadcast

# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = flights_df.join(broadcast(airports_df), \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan and compare against the original
broadcast_df.explain()

Comparing broadcast vs normal joins

You've created two types of joins, normal and broadcasted. Now your manager would like to know what the performance improvement is by using Spark optimizations. If the results are promising, you'll be given more opportunity to tweak the Spark setup as needed.

In [ ]:
# Don't execute, the airports data frame is not provided
start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = normal_df.count()
normal_duration = time.time() - start_time

start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))

## Complex processing and data pipelines
* Data pipelines are a set of steps to process data from source(s) to final output 
* Can consist of any number of steps or components
* Can span many systems  
  
What does a data pipeline look like?  
* Input(s), convert to dataframe:  
   - CSV, JSON, web services, databases
* Transformations: .withColumn(), .filter(), .drop()  
* Output(s): CSV, Parquet, database
* Validation
* Analysis  
  
Pipeline details:
* In spark, not formally a defined object
* Typically, all normal Spark Code requiered for task  



Quick pipeline

Before you parse some more complex data, your manager would like to see a simple pipeline example including the basic steps. For this example, you'll want to ingest a data file, filter a few rows, add an ID column to it, then write it out as JSON data.

The spark context is defined, along with the pyspark.sql.functions library being aliased as F as is customary.

In [41]:
import pyspark.sql.functions as F 
# Import the data to a DataFrame
departures_df = spark.read.csv("cleaning_data_with_pyspark_datasets/AA_DFW_2015_Departures_Short.csv", header=True)

# Remove any duration of 0
departures_df = departures_df.filter(~departures_df[3].contains('0'))

# Add an ID column
departures_df = departures_df.withColumn('ID', F.monotonically_increasing_id())
departures_df=departures_df.withColumnRenamed('Actual elapsed time (Minutes)','Duration')
departures_df = departures_df.withColumn('Duration', departures_df['Duration'].cast(IntegerType()))

# Write the file out to JSON format
departures_df.write.json('output_departures.json')

**Data handling techniques**
* Incorrect data: empty rows, commented lines, headers,...
* Nested structures: multiple delimeters
* Non-regular data  
  
Spark's CSV parser:  
* Automatically removes blank lines  
* Can remove comments using an optional argument  
df1=spark.read.csv('datafile.csv.gz',comment='#')
* Handles header fields: Defined via argument, ignored if a schema is defined  
df1=spark.read.csv('datafile.csv.gz', header=True)  


Removing commented lines

Your boss would like you to perform some complex parsing on a new dataset. The data represents annotation data for the ImageNet dataset, but focusing specifically on dog breeds and identifying them in images. Before any actual analysis can occur, you'll need to clear out several components of invalid / incorrect data. The general schema of the document is unknown so you'd like to import the rows into a single column, allowing for quick analysis.

To start, you need to remove all commented rows in the dataset.

In [ ]:
# Don't execute this chunk, the dataset is not provided
# Import the file to a DataFrame and perform a row count
annotations_df = spark.read.csv('annotations.csv.gz', sep='|')
full_count = annotations_df.count()

# Count the number of rows beginning with '#'
comment_count = annotations_df.where(col('_c0').startswith('#')).count()

# Import the file to a new DataFrame, without commented rows
no_comments_df = spark.read.csv('annotations.csv.gz', header=True, sep='|' ,comment='#')

# Count the new DataFrame and verify the difference is as expected
no_comments_count = no_comments_df.count()
print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, no_comments_count))

Removing invalid rows

Now that you've successfully removed the commented rows, you have received some information about the general format of the data. There should be at minimum 5 tab separated columns in the DataFrame. Remember that your original DataFrame only has a single column, so you'll need to split the data on the tab (\t) characters.

The DataFrame annotations_df is already available, with the commented rows removed. The spark.sql.functions library is available under the alias F. The initial number of rows available in the DataFrame is stored in the variable initial_count.

In [ ]:
# Split _c0 on the tab character and store the list in a variable
tmp_fields = F.split(annotations_df['_c0'], '\t')

# Create the colcount column on the DataFrame
annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# Remove any rows containing fewer than 5 fields
annotations_df_filtered = annotations_df.filter(~ (annotations_df['colcount']<5))

# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))

Splitting into columns

You've cleaned up your data considerably by removing the invalid rows from the DataFrame. Now you want to perform some further transformations by generating specific meaningful columns based on the DataFrame content.

In [ ]:
# Split the content of _c0 on the tab character (aka, '\t')
split_cols = F.split(annotations_df['_c0'], '\t')

# Add the columns folder, filename, width, and height
split_df = annotations_df.withColumn('folder', split_cols.getItem(0))
split_df = split_df.withColumn('filename', split_cols.getItem(1))
split_df = split_df.withColumn('width',split_cols.getItem(2))
split_df=split_df.withColumn('height',split_cols.getItem(3))

# Add split_cols as a column
split_df = split_df.withColumn('split_cols',split_cols)

Further parsing

You've molded this dataset into a significantly different format than it was before, but there are still a few things left to do. You need to prep the column data for use in later analysis and remove a few intermediary columns.

In [ ]:
def retriever(cols, colcount):
  # Return a list of dog data
  return cols[4:colcount]

# Define the method as a UDF
udfRetriever = F.udf(retriever, ArrayType(StringType()))

# Create a new column using your UDF
split_df = split_df.withColumn('dog_list', udfRetriever(split_df.split_cols, split_df.colcount))

# Remove the original column, split_cols, and the colcount
split_df = split_df.drop('_c0').drop('split_cols').drop('colcount')

**Data validation**  
Validation is:  
* Verifying that a dataset complies with the expected format
* Number of rows/columns
* Data types
* Complex validation rules. Using spark components to validate logic
   - calculations
   - verifying against an external source
   - likely uses a UDF to modify / verify the DataFrame 
  
Validation via joins:  
* Compares data against known values
* Easy to find data in a given set
* Comparatively fast  
* This automatically removes any rows with company not in valid_df


Broadcast join is an optimization technique in the PySpark SQL engine that is used to join two DataFrames. This technique is ideal for joining a large DataFrame with a smaller one. Traditional joins take longer as they require more data shuffling and data is always collected at the driver. In order to do broadcast join, we should use the broadcast shared variable.

In [ ]:
# Rename the column in valid_folders_df
valid_folders_df = valid_folders_df.withColumnRenamed('_c0', 'folder')

# Count the number of rows in split_df
split_count = split_df.count()

# Join the DataFrames
joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")

# Compare the number of rows remaining
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))

Examining invalid rows

You've successfully filtered out the rows using a join, but sometimes you'd like to examine the data that is invalid. This data can be stored for later processing or for troubleshooting your data sources.

You want to find the difference between two DataFrames and store the invalid rows.

In [ ]:
# Determine the row counts for each DataFrame
split_count = split_df.count()
joined_count = joined_df.count()

# Create a DataFrame containing the invalid rows
invalid_df = split_df.join(F.broadcast(joined_df), 'folder', 'left_anti')

# Validate the count of the new DataFrame is as expected
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

# Determine the number of distinct folder rows removed
invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)

**Final analysis and delivery**  
Analysis calculation (UDF).  
Inline calculations.      

Dog parsing

You've done a considerable amount of cleanup on the initial dataset, but now need to analyze the data a bit deeper. There are several questions that have now come up about the type of dogs seen in an image and some details regarding the images. You realize that to answer these questions, you need to process the data into a specific type. Before you can use it, you'll need to create a schema / type to represent the dog details.

In [ ]:
# Select the dog details and show 10 untruncated rows
print(joined_df.select('dog_list').show(10, truncate=False))

# Define a schema type for the details in the dog list
DogType = StructType([
	StructField("breed", StringType(), False),
    StructField("start_x", IntegerType(), False),
    StructField("start_y",IntegerType(),False),
    StructField("end_x",IntegerType(), False),
    StructField("end_y",IntegerType(),False)
])

Per image count

Your next task in building a data pipeline for this dataset is to create a few analysis oriented columns. You've been asked to calculate the number of dogs found in each image based on your dog_list column created earlier. You have also created the DogType which will allow better parsing of the data within some of the data columns.

In [ ]:
# Create a function to return the number and type of dogs as a tuple
def dogParse(doglist):
  dogs = []
  for dog in doglist:
    (breed, start_x, start_y, end_x, end_y) = dog.split(',')
    dogs.append((breed, int(start_x), int(start_y), int(end_x), int(end_y)))
  return dogs

# Create a UDF
udfDogParse = F.udf(dogParse, ArrayType(DogType))

# Use the UDF to list of dogs and drop the old column
joined_df = joined_df.withColumn('dogs', udfDogParse('dog_list')).drop('dog_list')

# Show the number of dogs in the first 10 rows
joined_df.select(F.size('dogs')).show(10)

Percentage dog pixels

The final task for parsing the dog annotation data is to determine the percentage of pixels in each image that represents a dog (or dogs). You'll need to use the various techniques you've learned in this course to help calculate this information and add it as columns for later analysis.

To calculate the percentage of pixels, first calculate the total number of pixels representing each dog then sum them for the image. You can calculate the bounding box with the formula:

(Xend - Xstart) * (Yend - Ystart)

NOTE: You can ignore the possibility of overlapping bounding boxes in this instance.

For the percentage, calculate the total number of "dog" pixels divided by the total size of the image, multiplied by 100.

In [ ]:
# Define a UDF to determine the number of pixels per image
def dogPixelCount(doglist):
  totalpixels = 0
  for dog in doglist:
    totalpixels += (dog[3] - dog[1]) * (dog[4] - dog[2])
  return totalpixels

# Define a UDF for the pixel count
udfDogPixelCount = F.udf(dogPixelCount, IntegerType())
joined_df = joined_df.withColumn('dog_pixels', udfDogPixelCount('dogs'))

# Create a column representing the percentage of pixels
joined_df = joined_df.withColumn('dog_percent', (joined_df.dog_pixels / (joined_df.width * joined_df.height)) * 100)

# Show the first 10 annotations with more than 60% dog
joined_df.where('dog_percent > 60').show(10)